1.Apple Store App 独立开发者，你要搞限时促销，为你的应用生成激活码（或者优惠券），使用 Python 如何生成 200 个激活码（或者优惠券）

In [199]:
import random
import os

from datetime import date

In [200]:
#优惠券规则:APP编号三位+年月日+随机7位数
#坑1：用的datetime.datetime会返回单位数的月份和日期
#坑2：有两种方式填充两位的月份和日期，格式化和字符串格式时间
cdate = date.today()
cyear = str(cdate.year)
cmonth = cdate.strftime('%m')
cday = cdate.strftime('%d')
app_number = '001'

In [202]:
end4 = []
n = 1 
while True:
    rand_end4 = random.randint(1000000,9999999)
    end4.append(app_number + cyear + cmonth + cday + str(rand_end4))
    n += 1
    if n > 200:
        break

In [203]:
if not os.path.exists('./store_quan.txt'):
    with open('store_quan.txt','wt') as f:
        f.write('优惠券编号\n')

In [204]:
with open('store_quan.txt','rt') as f:
    data = f.readlines()

In [205]:
#坑1：writelines写文件的时候是list或者迭代器 | 必须加入换行符号
with open('store_quan.txt','at') as f:
    for end in end4:
        if not end in data:        
            f.writelines(end + '\n')

2.将 0001 题生成的 200 个激活码（或者优惠券）保存到 MySQL 关系型数据库中

In [206]:
import pymysql

In [207]:
db = pymysql.connect(
    host='192.168.148.88', 
    port=3306,
    user='root', 
    passwd='123456', 
    db='exercise_100',
    charset='utf8')
cursor = db.cursor()

In [209]:
with open('store_quan.txt','rt') as f:
    data = f.readlines()

In [ ]:
#坑1：不会写mysql的插入语句
#坑2：mysql的语句必须用字符串格式化 | %i不需要用引号 | %i表示整数
#坑3：主键不能重复 | enumerate return index->int
for lineno, value in enumerate(data[2 : ], 2):
    store_num = value[ : 3]
    issue_date = value[3 : 11]
    random_num = value[11 : -1]
    #print(type(lineno), type(store_num), type(issue_date), type(random_num))
    #print(lineno, store_num, issue_date, random_num)
    sql = "INSERT INTO coupon (seqnum,store_num, issue_date, random_num) values (%i, '%s','%s','%s')" % (lineno,store_num, issue_date, random_num)
    cursor.execute(sql)
    db.commit()
    sql = 'select * from coupon'
    cursor.execute(sql)
    data = cursor.fetchone()
    print(data)

db.close()

3.将 0001 题生成的 200 个激活码（或者优惠券）保存到 Redis 非关系型数据库中

4.任一个英文的纯文本文件，统计其中的单词出现的个数

In [10]:
#坑1：TypeError: unhashable type: 'list'  <- content.append([])
#坑2：row.split()只用空格少分了太多次，例如，ok？、ok-ok 、ok,ok、ok.ok
import re

from collections import Counter

content = []

with open('exercise_4.txt', 'rt') as f:
    rows = f.readlines()
    for row in rows:
        for word in row.split():
            content.append(word)
            
c = Counter(content)
c.most_common(10)

[('you', 34),
 ('to', 33),
 ('the', 27),
 ('and', 24),
 ('that', 20),
 ('your', 16),
 ('a', 14),
 ('for', 12),
 ('it', 11),
 ('was', 11)]

In [ ]:
#坑1：\w = '[^A-Za-z0-9_]'
import re

from collections import Counter

with open('exercise_4.txt', 'rt') as f:
    content = f.read()
    content_list = [i for i in re.split(r'\W', content) if len(i) > 0]
    #content_list = re.split(r'\W', content)
    
c = Counter(content_list)
c.most_common(50)

In [ ]:
#坑1：考虑常用缩写
import re

from collections import Counter

with open('exercise_4.txt', 'rt') as f:
    content = f.read()
    content = re.sub("'re", " are", content)
    content = re.sub("doesn't", "does not", content)
    content = re.sub("hat's", "hat is", content)
    
    content_list = [i for i in re.split(r'\W', content) if len(i) > 0]
    #content_list = re.split(r'\W', content)
    
c = Counter(content_list)
c.most_common(20)

5.你有一个目录，装了很多照片，把它们的尺寸变成都不大于 iPhone5 分辨率的大小

In [ ]:
from PIL import Image

import os

path = r'D:\360MoveData\Users\86138\Desktop\jpg'
jpg_path = [i for i in os.listdir(path) if i.endswith('.jpg')]
jpg_path

# iphone5 分辨率 = 1136*640
#坑1：后缀名的变更.jpg  -> _md.jpg
#坑2：jpg[:-4] 不知道取到哪一位
#坑3：jpg_path没有带绝对路径，只有带文件名
os.chdir(r'D:\360MoveData\Users\86138\Desktop\jpg')

def produceImage(file_in, width, height, file_out):
    image = Image.open(file_in)
    resized_image = image.resize((width, height), Image.ANTIALIAS)
    resized_image.save(file_out)
    resized_image.close()

if __name__ == '__main__':
    for jpg in jpg_path:
        width = 640
        height = 1136
        file_out = jpg[:-4] + '_md.jpg'
        produceImage(jpg, width, height, file_out)

6.你有一个目录，放了你一个月的日记，都是 txt，为了避免分词的问题，

假设内容都是英文，请统计出你认为每篇日记最重要的词

与004类似，不做了

7.有个目录，里面是你自己写过的程序，统计一下你写过多少行代码。包括空行和注释，但是要分别列出来

列出所有是代码的文件

注释开头 #、''' '''、'""" """

In [47]:
#file.read-project,如果使用file.readlines，不好统计多行注释
#k：文件执行路径
#k：文件对象是迭代器，迭代一次耗尽
#k：局部作用域、全局作用域与嵌套函数对应使用，相当注意
#k：正则表达式的返回结果 re.findall(pat, str) -> len() judge
#k：if下各条件的组合 or and
#k：匹配多行注释
#k：正则表示式贪心和非贪心模式，注意太坑了
#k：
dir_path = r'D:\360MoveData\Users\86138\Desktop\myself'

import os
import re

pat1 = re.compile(r'\("""$') # beacause print(''' | print("""
pat2 = re.compile(r"\('''$")
pat3 = re.compile('("""|\'\'\')(.+?)("""|\'\'\')', re.S) # k (.+) 不加括号，匹配不到内容 -> [((), (), ()),....]
pat4 = re.compile('("""|\'\'\')(.+?)("""|\'\'\')') # k (.+) 不加括号，匹配不到内容 -> [((), (), ()),....]

os.chdir(dir_path) # 修改工作路径

py_file = [i for i in os.listdir(dir_path) if i.endswith('.py')]

raws_sum = 0 # 总代码行数
comment_sum = 0 # 总注释行数
blank_sum = 0 # 总空格行数
raws_all = 0 # 总行数
#dhzs = 0

for file in py_file:
    n = 0
    with open(os.path.join(dir_path,file), encoding='utf8') as f:
        # 弄清楚局部作用域和全局作用域
        # f 迭代器 ，一次调用就耗尽
        file_content = f.read()
        for i in re.findall(pat3, file_content):
            if '\n' in i[1].strip('\n'):
                n += i[1].strip('\n').count('\n') + 3
            else:
                n += 1
        
    with open(os.path.join(dir_path,file), encoding='utf8') as f:
        file_content = f.readlines()
        file_content = [ i.rstrip('\n') for i in file_content]
        file_content = [i.strip() for i in file_content]
        
        code_rows = len(file_content) # 每个文件的行数
        blank_space = 0 # 每个文件的空格行数
        comment = 0 # 每个文件的注释行数
        
        for i in file_content:
            if len(i) == 0:
                blank_space += 1
            elif i.startswith('#'):
                comment += 1
                
    comment = comment + n
    
    raws_all += code_rows
    comment_sum += comment
    blank_sum += blank_space
    raws_sum += code_rows - blank_space - comment
    
    print('当前文件：{} 文件行数{}  代码{}  注释{}-空格{}'.format(
            file, code_rows, code_rows - blank_space - comment, comment, blank_space))
    
print('全部文件行总和：{}  总代码行数：{}行  总注释行数：{}行  总空格行数：{}行'.format(
        raws_all, raws_sum, comment_sum, blank_sum))

当前文件：clusterMetricsEvaluate.py 文件行数89  代码54  注释30-空格5
当前文件：cmPlot.py 文件行数39  代码12  注释22-空格5
当前文件：commendOnItem.py 文件行数35  代码14  注释14-空格7
当前文件：conTelnet.py 文件行数56  代码29  注释12-空格15
当前文件：csv2pie.py 文件行数36  代码25  注释7-空格4
当前文件：plotClassifierRegions.py 文件行数58  代码38  注释16-空格4
当前文件：treeVisualization.py 文件行数25  代码14  注释8-空格3
全部文件行总和：338  总代码行数：186行  总注释行数：109行  总空格行数：43行


In [ ]:
 8.一个HTML文件，找出里面的正文

In [68]:
# doc                     https://docs.python.org/3/library/html.parser.html
file = 'D:/360MoveData/Users/86138/Desktop/lxf.html'

from html.parser import HTMLParser  

class MyHTMLParser(HTMLParser):   
    def __init__(self):   
        HTMLParser.__init__(self) #    
        self.links = []   
    def handle_starttag(self, tag, attrs):   
        #print "Encountered the beginning of a %s tag" % tag   
        if tag == "a":   
            if len(attrs) == 0:   
                pass   
            else:   
                for (variable, value) in attrs:   
                    if variable == "Boby":   
                        self.links.append(value)   
                     
if __name__ == "__main__":   
    hp = MyHTMLParser()
    f = open(file, encoding='utf8')
    fr = f.read()
    hp.feed(fr)  # must be str  
    hp.close()
    n = 0
    for i in set(hp.links):
        print(hp.links[1])

https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.li

In [ ]:
9.一个HTML文件，找出里面的链接

In [72]:
from lxml import etree

with open(file, encoding='utf8') as f:
    data = f.read()

selector = etree.HTML(data)

result = selector.xpath('//a/@href')
for each in result:
    print (each)

https://www.liaoxuefeng.com/wiki/1016959663602400/1017805396770016#0
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/
https://www.liaoxuefeng.com/blog/
https://www.liaoxuefeng.com/category/895882450960192
https://www.liaoxuefeng.com/category/895882413934592
https://www.liaoxuefeng.com/wiki/1252599548343744
https://www.liaoxuefeng.com/wiki/1016959663602400
https://www.liaoxuefeng.com/wiki/1022910821149312
https://www.liaoxuefeng.com/wiki/1177760294764384
https://www.liaoxuefeng.com/wiki/896043488029600
https://www.liaoxuefeng.com/discuss
https://www.liaoxuefeng.com/wiki/1016959663602400/1017805396770016#0
https://www.liaoxuefeng.com/category/895882450960192
https://www.liaoxuefeng.com/category/895882413934592
https://www.liaoxuefeng.com/wiki/1252599548343744
https://www.liaoxuefeng.com/wiki/1016959663602400
https://www.liaoxuefeng.com/wiki/1022910821149312
https://www.liaoxuefeng.com/wiki/1177760294764384
https://www.liaoxuefeng.com/wiki/896043488029600
https://www.liaoxuefeng.

In [ ]:
10.使用 Python 生成字母验证码图片

In [55]:
#K：chr 返回值是当前整数对应的 ASCII 字符
#K：相关知识 https://blog.csdn.net/icamera0/article/details/50762050
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import random

# 随机字母:
def rndChar():
    return chr(random.randint(65, 91))
# 随机颜色1:
def rndColor(): # (255, 255, 255)
    return (random.randint(64, 255), random.randint(64, 255), random.randint(64, 255))
# 随机颜色2:
def rndColor2():
    return (random.randint(32, 127), random.randint(32, 127), random.randint(32, 127))

# 240 x 60:
width = 240
height = 60
image = Image.new('RGB', (width, height), (255, 255, 255)) # Creates a new image with the given mode and size
# 创建Font对象:ImageFont模块定义了相同名称的类，即ImageFont类。
# 这个类的实例存储bitmap字体，用于ImageDraw类的text()方法 
font = ImageFont.truetype('C:/windows/fonts/Arial.ttf', 36) # 
# 创建Draw对象:
draw = ImageDraw.Draw(image)
# 填充每个像素:
for x in range(width):
    for y in range(height):
        draw.point((x, y), fill=rndColor())
# 输出文字:
for t in range(4):
    draw.text((60 * t + 10, 10), rndChar(), font=font, fill=rndColor2()) # 字体左上角坐标 要写的内容 字体填充 写5个要扩大图片尺寸
# 模糊:
image = image.filter(ImageFilter.BLUR)
#image.save('code.jpg', 'jpeg');
image.show('code.jpg')